# Nuestro primer Chatbot

In [10]:
# imports
from langchain_ollama.llms import OllamaLLM
import gradio as gr

In [15]:
# Definimos modelo de LLM
MODEL = 'llama3.2'

In [ ]:
# Llamada de prueba
OllamaLLM(model=MODEL).invoke("hello there!")

In [5]:
system_prompt = "You are a helpful assistant"

In [6]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]

    print("Historial actual:")
    print(history)
    print("Mensaje por procesar:")
    print(messages)

    # misma plantilla de siempre para obtener respuesta en forma de Stream
    result = ""
    for chunk in OllamaLLM(model=MODEL).stream(messages):
        result += chunk or ""
        yield result

In [ ]:
# el siguiente metodo para la interfaz funciona a partir de gradio 5.12, verifiquemos la version con este comando
!pip show gradio

## Gracias a Gradio, tan solo necesitamos una linea para tener todo configurado

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [12]:
system_message = "You are a helpful assistant in a technology store. You should try to gently encourage \
the customer to try items that are on sale. Laptops are 20% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a computer', \
you could reply something like, 'Wonderful - we have lots options - including several that are part of our sales evemt.'\
Encourage the customer to buy laptos if they are unsure what to get."

In [13]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    result = ""
    for chunk in OllamaLLM(model=MODEL).stream(messages):
        result += chunk or ""
        yield result

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [18]:
system_prompt += "\nIf the customer asks for graphic card, you should respond that graphic cards are not on sale today, \
but remind the customer to look at laptops!"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [20]:
def chat(message, history):

    relevant_system_message = system_prompt
    # implementacion basica para buscar agregar cierto contexto a partir de cierta condicion, solo es un ejemplo, para nada recomendado para ambiente de produccion
    if 'graphic card' or 'gpu' in message:
        relevant_system_message += " The graphic cards are not on sale today,s; if you are asked for them, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    result = ""
    for chunk in OllamaLLM(model=MODEL).stream(messages):
        result += chunk or ""
        yield result

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()